# Mecanisme de atenție și transformatoare

Un dezavantaj major al rețelelor recurente este că toate cuvintele dintr-o secvență au același impact asupra rezultatului. Acest lucru duce la performanțe sub-optime în cazul modelelor standard LSTM encoder-decoder pentru sarcini de tip secvență-la-secvență, cum ar fi Recunoașterea Entităților Numite și Traducerea Automată. În realitate, anumite cuvinte din secvența de intrare au adesea un impact mai mare asupra ieșirilor secvențiale decât altele.

Să luăm în considerare un model de tip secvență-la-secvență, cum ar fi traducerea automată. Acesta este implementat prin două rețele recurente, unde o rețea (**encoder**) comprimă secvența de intrare într-o stare ascunsă, iar cealaltă, **decoder**, desfășoară această stare ascunsă într-un rezultat tradus. Problema cu această abordare este că starea finală a rețelei întâmpină dificultăți în a-și aminti începutul unei propoziții, ceea ce duce la o calitate slabă a modelului pentru propoziții lungi.

**Mecanismele de atenție** oferă o modalitate de a pondera impactul contextual al fiecărui vector de intrare asupra fiecărei predicții de ieșire a RNN-ului. Acest lucru este implementat prin crearea unor scurtături între stările intermediare ale RNN-ului de intrare și RNN-ului de ieșire. Astfel, atunci când generăm simbolul de ieșire $y_t$, vom lua în considerare toate stările ascunse de intrare $h_i$, cu coeficienți de greutate diferiți $\alpha_{t,i}$.

![Imagine care arată un model encoder/decoder cu un strat de atenție aditiv](../../../../../translated_images/ro/encoder-decoder-attention.7a726296894fb567.webp)
*Modelul encoder-decoder cu mecanism de atenție aditiv din [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citat din [acest articol de blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matricea de atenție $\{\alpha_{i,j}\}$ ar reprezenta gradul în care anumite cuvinte de intrare contribuie la generarea unui cuvânt dat în secvența de ieșire. Mai jos este un exemplu al unei astfel de matrice:

![Imagine care arată o aliniere exemplară găsită de RNNsearch-50, preluată din Bahdanau - arviz.org](../../../../../translated_images/ro/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figura preluată din [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Mecanismele de atenție sunt responsabile pentru o mare parte din starea actuală sau aproape actuală a artei în procesarea limbajului natural. Totuși, adăugarea atenției crește semnificativ numărul de parametri ai modelului, ceea ce a dus la probleme de scalare cu RNN-urile. O constrângere cheie a scalării RNN-urilor este că natura recurentă a modelelor face dificilă gruparea și paralelizarea antrenării. Într-un RNN, fiecare element al unei secvențe trebuie procesat în ordine secvențială, ceea ce înseamnă că nu poate fi paralelizat cu ușurință.

Adoptarea mecanismelor de atenție combinată cu această constrângere a dus la crearea modelelor Transformatoare, care reprezintă acum starea de artă, pe care le cunoaștem și le folosim astăzi, de la BERT la OpenGPT3.

## Modele transformatoare

În loc să transmită contextul fiecărei predicții anterioare în pasul următor de evaluare, **modelele transformatoare** folosesc **codificări poziționale** și atenție pentru a captura contextul unei intrări date într-o fereastră de text furnizată. Imaginea de mai jos arată cum codificările poziționale cu atenție pot captura contextul într-o fereastră dată.

![GIF animat care arată cum sunt realizate evaluările în modelele transformatoare.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Deoarece fiecare poziție de intrare este mapată independent la fiecare poziție de ieșire, transformatoarele pot paraleliza mai bine decât RNN-urile, ceea ce permite modele de limbaj mult mai mari și mai expresive. Fiecare cap de atenție poate fi utilizat pentru a învăța relații diferite între cuvinte, ceea ce îmbunătățește sarcinile de procesare a limbajului natural.

**BERT** (Bidirectional Encoder Representations from Transformers) este o rețea transformatoare foarte mare, cu mai multe straturi: 12 straturi pentru *BERT-base* și 24 pentru *BERT-large*. Modelul este mai întâi pre-antrenat pe un corpus mare de date text (Wikipedia + cărți) folosind antrenare nesupravegheată (prezicerea cuvintelor mascate într-o propoziție). În timpul pre-antrenării, modelul absoarbe un nivel semnificativ de înțelegere a limbajului, care poate fi apoi valorificat cu alte seturi de date prin ajustare fină. Acest proces se numește **învățare transferabilă**.

![Imagine de pe http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ro/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Există multe variații ale arhitecturilor Transformatoare, inclusiv BERT, DistilBERT, BigBird, OpenGPT3 și altele, care pot fi ajustate fin. Pachetul [HuggingFace](https://github.com/huggingface/) oferă un depozit pentru antrenarea multora dintre aceste arhitecturi cu PyTorch.

## Utilizarea BERT pentru clasificarea textului

Să vedem cum putem folosi modelul BERT pre-antrenat pentru a rezolva sarcina noastră tradițională: clasificarea secvențelor. Vom clasifica setul nostru de date original AG News.

Mai întâi, să încărcăm biblioteca HuggingFace și setul nostru de date:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Deoarece vom folosi un model BERT pre-antrenat, va fi necesar să utilizăm un tokenizer specific. Mai întâi, vom încărca un tokenizer asociat cu modelul BERT pre-antrenat.

Biblioteca HuggingFace conține un depozit de modele pre-antrenate, pe care le poți utiliza doar specificând numele lor ca argumente pentru funcțiile `from_pretrained`. Toate fișierele binare necesare pentru model vor fi descărcate automat.

Totuși, în anumite situații, va trebui să încarci propriile modele, caz în care poți specifica directorul care conține toate fișierele relevante, inclusiv parametrii pentru tokenizer, fișierul `config.json` cu parametrii modelului, greutățile binare etc.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Obiectul `tokenizer` conține funcția `encode` care poate fi utilizată direct pentru a codifica textul:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Apoi, să creăm iteratori pe care îi vom folosi în timpul antrenamentului pentru a accesa datele. Deoarece BERT folosește propria funcție de codificare, va trebui să definim o funcție de umplere similară cu `padify` pe care am definit-o anterior:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

În cazul nostru, vom folosi modelul BERT pre-antrenat numit `bert-base-uncased`. Să încărcăm modelul folosind pachetul `BertForSequenceClassfication`. Acest lucru asigură că modelul nostru are deja o arhitectură necesară pentru clasificare, inclusiv clasificatorul final. Veți vedea un mesaj de avertizare care indică faptul că greutățile clasificatorului final nu sunt inițializate, iar modelul ar necesita pre-antrenare - acest lucru este perfect în regulă, deoarece exact asta urmează să facem!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Acum suntem gata să începem antrenamentul! Deoarece BERT este deja pre-antrenat, dorim să începem cu o rată de învățare destul de mică pentru a nu distruge greutățile inițiale.

Toată munca grea este realizată de modelul `BertForSequenceClassification`. Când apelăm modelul pe datele de antrenament, acesta returnează atât pierderea, cât și ieșirea rețelei pentru minibatch-ul de intrare. Folosim pierderea pentru optimizarea parametrilor (`loss.backward()` efectuează trecerea înapoi), iar `out` pentru calcularea acurateței antrenamentului prin compararea etichetelor obținute `labs` (calculate folosind `argmax`) cu etichetele așteptate `labels`.

Pentru a controla procesul, acumulăm pierderea și acuratețea pe parcursul mai multor iterații și le afișăm la fiecare ciclu de antrenament `report_freq`.

Acest antrenament va dura probabil destul de mult timp, așa că limităm numărul de iterații.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Poți observa (mai ales dacă mărești numărul de iterații și aștepți suficient de mult) că clasificarea cu BERT ne oferă o acuratețe destul de bună! Acest lucru se datorează faptului că BERT înțelege deja foarte bine structura limbajului, iar noi trebuie doar să ajustăm clasificatorul final. Totuși, deoarece BERT este un model mare, întregul proces de antrenare durează mult și necesită o putere de calcul considerabilă! (GPU, și de preferat mai multe).

> **Note:** În exemplul nostru, am folosit unul dintre cele mai mici modele BERT pre-antrenate. Există modele mai mari care probabil vor oferi rezultate mai bune.


## Evaluarea performanței modelului

Acum putem evalua performanța modelului nostru pe setul de date de testare. Bucla de evaluare este destul de similară cu bucla de antrenare, dar nu trebuie să uităm să comutăm modelul în modul de evaluare apelând `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Concluzii

În această unitate, am văzut cât de ușor este să luăm un model de limbaj pre-antrenat din biblioteca **transformers** și să-l adaptăm pentru sarcina noastră de clasificare a textului. În mod similar, modelele BERT pot fi utilizate pentru extragerea entităților, răspunsul la întrebări și alte sarcini de procesare a limbajului natural.

Modelele Transformer reprezintă stadiul actual de vârf în NLP, iar în majoritatea cazurilor ar trebui să fie prima soluție cu care începeți să experimentați atunci când implementați soluții personalizate de NLP. Totuși, înțelegerea principiilor de bază ale rețelelor neuronale recurente discutate în acest modul este extrem de importantă dacă doriți să construiți modele neuronale avansate.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
